In [ ]:
import openai
import os
import sys
from io import StringIO

openai.api_key = os.environ["OPENAI_API_KEY"]


def chat_with_gpt_4(role, content):
    response = openai.Completion.create(
        engine="gpt-4", messages=[{"role": role, "text": content}], temperature=0.7
    )
    return response.choices[0].message["content"]


def run_python_code(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    try:
        exec(code)
    except Exception as e:
        print(e)
    sys.stdout = old_stdout
    return redirected_output.getvalue()


intro_message = """

"""
message_history = [{"role": "user", "text": intro_message}]

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    message_history.append({"role": "user", "content": user_input})
    response = chat_with_gpt_4(message_history)
    print("GPT-4: ", response)

    message_history.append({"role": "assistant", "content": response})

    if response.lower().find("user run this code:"):
        input("\n Want to run this code?")
        result = run_python_code(code)
        print("result: ", result)

        # share result with gpt
        message_history.append({"role": "user", "content": result})
        response = chat_with_gpt_4(message_history)
        print("GPT-4: ", response)
        message_history.append({"role": "assistant", "content": response})